In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

from tqdm.auto import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/neurips-adc-challenge-2024-calibrated-tfrecord/primary-neurips-dataset-20241015205628-c1d1f1m1b30.tfrecords


In [2]:
primary_dataset = tf.data.TFRecordDataset(
    "/kaggle/input/neurips-adc-challenge-2024-calibrated-tfrecord/primary-neurips-dataset-20241015205628-c1d1f1m1b30.tfrecords")

## Decode`tf.train.Examples` in the `TFRecord`s

In [3]:
def decode_fn(proto):
    features_formats = {
        "id": tf.io.FixedLenFeature([], tf.int64),
        "airs": tf.io.FixedLenFeature([], tf.string),
        "fgs": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(proto, features_formats)
    pid = example["id"]
    airs = tf.io.parse_tensor(example["airs"], tf.float64)
    fgs = tf.io.parse_tensor(example["fgs"], tf.float64)
    target = tf.io.parse_tensor(example["target"], tf.float64)
    return pid, airs, fgs, target

In [4]:
def concatenate_signals(proto):
    pid, airs, fgs, target = decode_fn(proto)
    signal = np.concatenate(
        [airs.numpy(), fgs.numpy().sum(axis=2)[:,:, np.newaxis,:]], axis=2
    )
    signal = tf.convert_to_tensor(signal)
    return pid, signal, target

In [5]:
dataset = primary_dataset.prefetch(
    buffer_size=tf.data.AUTOTUNE).map(lambda example: tf.py_function(
    func=concatenate_signals, inp=[example],
    Tout=[
        tf.TensorSpec(shape=None, dtype=tf.int64),
        tf.TensorSpec(shape=None, dtype=tf.float64),
        tf.TensorSpec(shape=None, dtype=tf.float64)
    ]))

In [6]:
np_signal_dataset = None
np_target_dataset = None
for obs in tqdm(iter(dataset)):
    try:
        if np_signal_dataset is None:
            np_signal_dataset = obs[1].numpy()
            np_target_dataset = obs[2].numpy()
        else:
            np_signal_dataset = np.concatenate([np_signal_dataset, obs[1].numpy()])
            np_target_dataset = np.concatenate([np_target_dataset, obs[2].numpy()])
    except ValueError:
        pass

0it [00:00, ?it/s]